---

## Gradient Descent

![](https://images.velog.io/images/qsdcfd/post/583ecaa0-f28c-4b01-ad6a-64c4bc9ed632/image.png)

Graient descent는 gradient를 활용해서 더 작은 loss를 갖는 w를 찾는 방식입니다.


### Hyperparameter

- W(weight) 초기화

- W를 찾는 횟수

- step 시 이동할 거리(learning rate), lr이 높으면 최적의 W를 빨리 찾지만 overshoot문제가 생기고 lr이 낮으면 오래 걸립니다.

즉, W의 변화량과 gradient는 비례 관계입니다.

### Batch Gradient Descent


![](https://images.velog.io/images/qsdcfd/post/953e6179-f961-42a4-b51b-fdc551092abd/image.png)

사실 loss가 데이터의 합으로 이루어지는 것처럼 gradient도 데이터의 합으로 있기에 데이터가 많을 수록 비용이 증가합니다. 

## SGD(Stochastic Gradient Descent)


말했다시피, 많은 수록 비용이 증가하므로 gradient descent를 변형시켜서 사용하는데 그중 대표적인 것이 SGD입니다.

![](https://images.velog.io/images/qsdcfd/post/0e498070-ea1f-47ce-b85c-b427da91c609/image.png)


SGD는 데이터에 대해 Gradient와 Loss를 구하는 것에 집중하지 않고 적은 수의 데이터로 구합니다. (* 적은 수의 데이터: minibatch, size:32,64,128)


*표본 데이터를 통해서 Gradient와 Loss를 구하므로, 표본 데이터의 분포를 설정할 때 전체 데이터와 유사해야합니다.*

즉, 손실함수의 기울기를 계산하여서 이 기울기 값에 학습률(Learning Rate)을 계산 값을 통해서 기존의 가중치 값을 갱신합니다.



- Code

class SGD:
    def __init__(self, lr=0.01):
        self.lr = lr
    
    def update(self, params, grads):
        for key in params.keys():
            params[key] -= self.lr * grads[key]


출처: https://sacko.tistory.com/42 [데이터 분석하는 문과생, 싸코]


### Hyperparameter

- minibatch

  
  - batch size: minibatch의 크기, GPU의 메모리 크기가 허용하는 한 가장 큰 게 좋습니다
   
   
   - Data sampling: 데이터 선정 방식, ranking이나 prediction에서 중요합니다.


![](https://images.velog.io/images/qsdcfd/post/91836a91-c671-4028-8de5-6b93c65ee80f/image.png)



### Problems
<br>

#### Overshoot


![](https://images.velog.io/images/qsdcfd/post/bbfa049a-2516-4c1e-95d1-41ed7717314c/image.png)

기울기가 과하게 진동하면서 더 많은 step이 소요되는 것을 말합니다.

<br>

#### Local Minimum


![](https://images.velog.io/images/qsdcfd/post/6285c02c-6de2-44cf-a0d0-e902a53899a2/image.png)


Local minima 문제는 에러를 최소화시키는 최적의 파라미터를 찾는 문제로 위의 그림처럼 hole에 빠져서 global minimum을 찾기 힘들게 되는 문제입니다. 


*hole은 gradient =0 인 saddle point에 위치하는 점으로 학습이 지연이 됩니다.*


<br>

#### Noise


SGD는 적은 데이터로만 학습을 진행하므로 손실함수의 가중치의 방향이 항상 LOSS를 따르면 좋겠지만 노이즈가 껴서 그렇지 않은 경우도 당연히 있습니다.


![](https://images.velog.io/images/qsdcfd/post/79949866-031b-4361-804c-9e704f029a8d/image.png)


<br>

---

### SGD + Momentum


![](https://images.velog.io/images/qsdcfd/post/d4835b91-2570-4b6a-a507-59fb6098bb24/image.png)


- step의 정보를 반영하여 velocity와 gradient가 합쳐서 다음이 정해집니다.

- 딥러닝 모델에 잘 쓰입니다.

- Momentum은 SGD의 진행 추세(관성)입니다.

![](https://images.velog.io/images/qsdcfd/post/59b0d0ae-e573-43cb-b9c3-322d503c4c4b/image.png)

**관성으로 인해 W가 local minimum or saddle point문제가 생겨도 탈출할 수 있고 기울기가 지그재그로 움직이게 되는 것도 완화해줍니다.**

- decay rate: hyperparameter로 비율 감소의 의미를 나타냅니다.

- 정헤진 form보다는 상황에 따라 바뀔 수 있답니다.


### Nesterov Momentum


- 현재 위치에서 속도만큼 이동하는 동시에 gradient를 이용하여 다음 step을 정합니다.


- 직관적으로 사용하면 사용범위가 넓지 않아서 변형해서 적용을 많이 합니다.

![](https://images.velog.io/images/qsdcfd/post/3f6ee2f0-323b-40df-8b3e-db38a7af79ff/image.png)


#### Momentum Code

class Momentum:
    def __init__(self, lr=0.01, momentum=0.9):
        self.lr = lr
        self.momentum = momentum
        self.v = None
    
    def update(self, params, grads):
        if self.v is None:
            self.v = {}
            for key, val in params.items():
                self.v[key] = np.zeros_like(val)
        
        for key in params.keys():
            self.v[key] = self.momentum * self.v[key] 


출처: https://sacko.tistory.com/42 [데이터 분석하는 문과생, 싸코]

---

### AdaGrad

![](https://images.velog.io/images/qsdcfd/post/ca6f4af8-9f63-4d26-9b45-5921bd713ccd/image.png)

*AdaGrad는 Adaptive Gradient의 줄임말로서 지금까지 많이 변화한 매개변수는 적게 변화하도록, 반대로 적게 변화한 매개변수는 많이 변화하도록 learning late의 값을 조절하는 개념을 기존의 SGD에 적용한 것입니다. -위키백과-*


- Gradient가 크면 step size가 작아집니다.(큰 값으로 나누기 때문입니다.)

- Gradient가 작으면 step size가 커집니다.(작은 값으로 나누기 때문입니다.)

- 무수한 진동이 일어나는 문제를 완화시킵니다.

- 주의할 점은 step이 많이 진행될수록 grad_squared가 커지기에 GD가 멈추게 됩니다.


- Code

class AdaGrad:
    def __init__(self, lr=0.01):
        self.lr = lr
        self.h = None
    
    def update(self, params, grads):
        if self.h is None:
            self.h = {}
            for key, val in params.items():
                self.h[key] = np.zeros_like(val)
                
        for key in params.keys():
            self.h[key] += grads[key] * grads[key]
            params[key] -= self.lr * grads[key] / (np.sqrt(self.h[key]) + 1e-7)


출처: https://sacko.tistory.com/42 [데이터 분석하는 문과생, 싸코]



### RMSProp(AdaGrad 개선)

![](https://images.velog.io/images/qsdcfd/post/babe937b-f2e1-4857-bd3b-5212e150bf84/image.png)

RMSProp는 새로운 기울기의 정보만 반영하도록 해서 학습률이 크게 떨어지게 되어 0에 수렴하는 것을 방지합니다.




#### overshoot 완화


RMSProp는 step size가 점점 줄어들기에 기존의 SGD+ Momentum보다 덜합니다.

![](https://images.velog.io/images/qsdcfd/post/4302b60b-bbb0-4fd5-ab2a-f3fd192c3455/image.png)



### Adam

![](https://images.velog.io/images/qsdcfd/post/00163a9e-7eb3-444a-80da-8d8029656998/image.png)


- Adaptive Moment Estimation의 줄임말로 SGD + Momentum and RMSProp의 장점을 합친 방식입니다.

![](https://images.velog.io/images/qsdcfd/post/709db943-23b8-473f-b609-4ba1b9997139/image.png)


#### Hyperparameter



- beta1=0.9

- beta2는 1의 가까울 수록 t=0이 되어 step size가 커지므로 Bias correction을 추가하여 사용한다.

- lr = 1e-3 , 5e-4, 1e-4


#### 표를 이용한 총 정리

![](https://images.velog.io/images/qsdcfd/post/4cac0cde-b6c7-4d43-8e14-3b5bef993295/image.png)